TODO
- test aoi load on multipolygon feature?

In [1]:
import os
from pathlib import Path

import psycopg2
import geopandas as gpd
from shapely import wkt

from secret_codes import *

In [2]:
# Create work path to area of interest
data_path = os.path.join(str(Path.home()), "Desktop", "forest_mon", "data")

# Check to see if path exists
os.path.exists(data_path)

True

In [3]:
# Load AoI 
aoi = gpd.read_file(os.path.join(data_path, "gardens_edit.json"))

In [7]:
# Check for multipolygon
if aoi.shape[0] > 1:
    print("Check for multipolygon")
elif aoi.shape[0] == 0:
    print("Empty geometry")

In [8]:
# Get AoI geometry as string
aoi_geom_string = wkt.dumps(aoi.geometry[0])

In [9]:
# Create connection to test database
conn = psycopg2.connect(user="postgres",
                        password=PG_PASS,
                        host="localhost",
                        port=5432,
                        database="forest_test")

# Create cursor
cur = conn.cursor()

In [23]:
# User enters forest name
insert_forest_name = 'test_forest'

In [12]:
### Test insert values
insert_aoi_query = "INSERT INTO %s (forest_name, is_active, date_planted) VALUES (%%s, %%s, %%s)" % 'forest_info'

cur.execute(insert_aoi_query, (insert_forest_name, 'TRUE', '2020-05-01'))

conn.commit()

In [14]:
# Update with geom
update_geom_query = """UPDATE %s
                    SET geom = st_geomfromtext(%%s, 4326)
                    WHERE forest_name = %%s """ % 'forest_info'

cur.execute(update_geom_query, (aoi_geom_string, insert_forest_name))

conn.commit()

In [ ]:
#cur.execute("ROLLBACK;")

In [32]:
"""'test_forest' string should be var"""

select_forest_id = "SELECT forest_id, forest_name FROM %s WHERE forest_name = 'test_forest'" % 'forest_info'

cur.execute(select_forest_id)

results = cur.fetchall()

for row in results:
    insert_forest_id = row[0]
    print(f"Forest id: {insert_forest_id}")
    print(f"Forest name: {row[1]}")

Forest id: 5
Forest name: test_forest


In [33]:
# Insert data into locations table

insert_loc_query = "INSERT INTO %s (forest_id, loc_sys, loc_val) VALUES (%%s, %%s, %%s)" % 'locations'

cur.execute(insert_loc_query, (insert_forest_id, 'WRS', '149,38'))


In [34]:
conn.commit()

In [28]:
cur.execute("ROLLBACK;")